# Convert from .pt to .onnx

In [13]:
import torch
from train import ClassifierModel
# import torch._dynamo

DUMMY_PARAMS =  {
    "architect_settings" : {
            "task": "None",
            "name": "model-test",
            "backbone": {
                    "name": "mobilenet-s",
                    "is_full": False,
                    "is_pretrained": True,
                    "is_freeze": False, 
            },
            "n_cls": 2,
            },
    "dataset_settings": {
            
            },
    "training_settings":{
        'metric': 'accuracy',
    }
}

torch_model = ClassifierModel(DUMMY_PARAMS).load_from_checkpoint(r".neptune/Untitled/AIS-170/checkpoints/best_034.ckpt").cpu()
torch_model = torch_model.model
torch_input = torch.randn(1, 3, 224, 224)
torch_out = torch_model(torch_input)
# Export the model
torch.onnx.export(torch_model,               # model being run
                  torch_input,                         # model input (or a tuple for multiple inputs)
                  "test.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

# Check .onnx model

In [10]:
import onnx
import onnxruntime
onnx_model = onnx.load("test.onnx")
onnx.checker.check_model(onnx_model)

import numpy as np

ort_session = onnxruntime.InferenceSession("test.onnx", providers=["CPUExecutionProvider"])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(torch_input)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

# Using PyTorch Lightning

In [1]:
from train import ClassifierModel
import torch

DUMMY_PARAMS =  {
    "architect_settings" : {
            "task": "None",
            "name": "model-test",
            "backbone": {
                    "name": "mobilenet-s",
                    "is_full": False,
                    "is_pretrained": True,
                    "is_freeze": False, 
            },
            "n_cls": 2,
            },
    "dataset_settings": {
            
            },
    "training_settings":{
        'metric': 'accuracy',
    }
}

torch_model = ClassifierModel(DUMMY_PARAMS).load_from_checkpoint(r".neptune/Untitled/AIS-170/checkpoints/best_034.ckpt").cpu()
torch_input = torch.randn(1, 3, 224, 224)
filepath = "model.onnx"
torch_model.to_onnx(filepath, torch_input, export_params=True,  
                    input_names = ['input'],   # the model's input names
                  output_names = ['output']) # the model's output names

c:\Users\197796\Anaconda3\envs\AISeed\lib\site-packages\neptune\common\warnings.py:62: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  warnings.warn(


In [5]:
import onnx
import onnxruntime
onnx_model = onnx.load("model.onnx")
onnx.checker.check_model(onnx_model)

import numpy as np

torch_out = torch_model(torch_input)

ort_session = onnxruntime.InferenceSession("model.onnx", providers=["CPUExecutionProvider"])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(torch_input)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from test.onnx failed:Load model test.onnx failed. File doesn't exist